In [1]:
# Notebook setup: ensure we run from repo root (so relative paths like configs/ and runs/ work)
import os
import sys
from pathlib import Path

def _find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for cand in [p] + list(p.parents):
        if (cand / "pyproject.toml").exists() and (cand / "src").exists():
            return cand
    # Fallback: if executed from notebooks/, go one level up
    if p.name.lower() == "notebooks" and (p.parent / "src").exists():
        return p.parent
    return p

REPO_ROOT = _find_repo_root()
os.chdir(REPO_ROOT)

src_path = REPO_ROOT / "src"
if src_path.exists() and str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("Repo root:", REPO_ROOT)

# Note:
# Set RUN_ID below to an existing run you created via the CLI (e.g., RUN_XXX_0005).
RUN_ID = "RUN_2026"

Repo root: C:\Users\Martín\Desktop\inkswarm-core\usul-inkswarm-detectlab


# 01 — Placeholder data inspection

In [2]:
from pathlib import Path
import pandas as pd

from inkswarm_detectlab.io.paths import raw_table_basepath
from inkswarm_detectlab.io.tables import read_auto

# Set this to an existing run you created via the CLI (e.g., RUN_XXX_0005)
#RUN_ID = "RUN_XXX_0006"  # <-- change me
run_dir = Path("runs") / RUN_ID

def _read_required_table(event: str):
    base = raw_table_basepath(run_dir, event)
    try:
        return read_auto(base)
    except FileNotFoundError as e:
        print(f"ERROR: could not find raw parquet for event={event!r} under {run_dir}")
        if run_dir.exists():
            print("Found files under run_dir/raw/:")
            raw_dir = run_dir / "raw"
            if raw_dir.exists():
                for p in sorted(raw_dir.glob("*")):
                    print(" -", p)
            else:
                print(" - (no raw/ folder found)")
        else:
            print("Run directory does not exist:", run_dir)
            print("Tip: make sure you launched Jupyter from the repo root, or use the setup cell above.")
        raise

login = _read_required_table("login_attempt")
checkout = _read_required_table("checkout_attempt")

print("login shape:", login.shape)
display(login.head())

print("checkout shape:", checkout.shape)
display(checkout.head())


login shape: (10891, 27)


,run_id,event_id,event_ts,user_id,session_id,ip_hash,device_fingerprint_hash,country,is_fraud,label_replicators,...,mfa_used,mfa_result,support_contacted,support_channel,support_responder_type,support_wait_seconds,support_handle_seconds,support_cost_usd,support_resolution,support_offset_seconds
0,RUN_2026,login_0000000001,2025-12-01 00:00:49-03:00,user_00035,sess_84600,ip_1915,dev_0242,AR,True,True,...,False,not_applicable,False,none,none,NaN,NaN,NaN,none,NaN
1,RUN_2026,login_0000000002,2025-12-01 00:01:41-03:00,user_00101,sess_19375,ip_8295,dev_0585,AR,True,True,...,False,not_applicable,False,none,none,NaN,NaN,NaN,none,NaN
2,RUN_2026,login_0000000003,2025-12-01 00:03:37-03:00,user_00028,sess_29751,ip_0600,dev_9462,AR,False,False,...,False,not_applicable,False,none,none,NaN,NaN,NaN,none,NaN
3,RUN_2026,login_0000000004,2025-12-01 00:05:06-03:00,user_00035,sess_84600,ip_1915,dev_0242,AR,False,False,...,True,pass,False,none,none,NaN,NaN,NaN,none,NaN
4,RUN_2026,login_0000000005,2025-12-01 00:05:28-03:00,user_00061,sess_57557,ip_2782,dev_1364,AR,False,False,...,False,not_applicable,False,none,none,NaN,NaN,NaN,none,NaN


checkout shape: (1116, 17)


,run_id,event_id,event_ts,user_id,session_id,ip_hash,device_fingerprint_hash,country,is_fraud,metadata_json,payment_value,basket_size,is_first_time_user,is_premium_user,credit_card_hash,checkout_result,decline_reason
0,RUN_2026,checkout_0000000001,2025-12-01 00:07:24-03:00,user_00037,sess_86113,ip_5250,dev_7638,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",9.041241,6,False,True,cc_156130,success,None
1,RUN_2026,checkout_0000000002,2025-12-01 00:21:49-03:00,user_00015,sess_74348,ip_3721,dev_3154,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",35.095917,2,True,False,cc_194069,success,None
2,RUN_2026,checkout_0000000004,2025-12-01 00:31:44-03:00,user_00154,sess_68123,ip_5519,dev_3122,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",13.368612,4,False,False,cc_097440,success,None
3,RUN_2026,checkout_0000000003,2025-12-01 00:33:09-03:00,user_00063,sess_99814,ip_9885,dev_6442,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",5.485563,3,True,False,cc_059075,success,None
4,RUN_2026,checkout_0000000005,2025-12-01 01:31:49-03:00,user_00003,sess_42858,ip_4713,dev_0149,AR,False,"{""campaign_id"":null,""note"":""fraud labels disab...",7.335919,2,False,False,None,success,None
